In [7]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [ ]:
# filepath = os.getcwd() + '/event_data'
filepath = os.getcwd() + '/data'


print(filepath)

for root,dir,file in os.walk(filepath):
    list_files_path = glob.glob(os.path.join(root,'*'))

print(list_files_path)


In [ ]:
data_lines = []

for file in list_files_path:
    with open(file,'r',encoding='utf-8',newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        for line in csvreader:
            print(line)
            data_lines.append(line)



csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in data_lines:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        

In [21]:
from cassandra.cluster import Cluster 

cluster = Cluster()
try:
    session = cluster.connect()
except Exception as e :
    print(e)


In [22]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifydatabase
        WITH REPLICATION = 
        {'class' : 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print (e)

In [23]:
try:
    session.set_keyspace('sparkifydatabase')
except Exception as e:
    print (e)

In [24]:
create_table_1 = "CREATE TABLE IF NOT EXISTS song_details"
create_table_1 = create_table_1 + (""" (session_id INT, item_in_session INT, artist TEXT, song_title TEXT,
                                        song_length FLOAT, PRIMARY KEY (session_id, item_in_session)
                )""")
try:
    session.execute(create_table_1)
except Exception as e:
    print(e)
    
select_query_1 = """ SELECT artist, song_title, song_length FROM song_details
                    WHERE session_id = %s 
                    AND item_in_session = %s 
                    """

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_details(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [28]:
try:
    rows = session.execute(select_query_1, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song_title, "\nSong Length: "+str(row.song_length))

Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875


In [29]:
create_table_2 = "CREATE TABLE IF NOT EXISTS artist_details"
create_table_2 = create_table_2 + (""" (user_id INT, session_id INT, item_in_session INT, 
                                        artist_name TEXT, song_title TEXT, user_first_name TEXT, 
                                        user_last_name TEXT, PRIMARY KEY (user_id, session_id, item_in_session)
                )""")
try:
    session.execute(create_table_2)
except Exception as e:
    print(e)
    
select_query_2 = """ SELECT artist_name, song_title, user_first_name, user_last_name FROM artist_details
                    WHERE user_id = %s
                    AND session_id = %s 
                """

In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_details(user_id, session_id, item_in_session, artist_name, \
                                            song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [33]:
try:
    rows = session.execute(select_query_2, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist_name, "\nSong Title: "+row.song_title, "\nUser First Name: "+row.user_first_name, "\nUser Last Name: "+row.user_last_name)

Artist: Down To The Bone 
Song Title: Keep On Keepin' On 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Three Drives 
Song Title: Greece 2000 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Sebastien Tellier 
Song Title: Kilometer 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Lonnie Gordon 
Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
User First Name: Sylvie 
User Last Name: Cruz


In [34]:
create_table_3 = "CREATE TABLE IF NOT EXISTS user_details"
create_table_3 = create_table_3 + (""" (song_title TEXT, user_id INT, user_first_name TEXT, user_last_name TEXT, 
                                        PRIMARY KEY ((song_title), user_id)
                )""")
try:
    session.execute(create_table_3)
except Exception as e:
    print(e)

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_details(song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [36]:
select_query_3 = """ SELECT user_first_name, user_last_name FROM user_details WHERE song_title = %s;"""
print(select_query_3)

 SELECT user_first_name, user_last_name FROM user_details WHERE song_title = %s;


In [ ]:
try:
    rows = session.execute(select_query_3, ('All Hands Against His Own', ))
except Exception as e:
    print(e)

for row in rows:
    print("User First Name: "+row.user_first_name, "\nUser Last Name: "+row.user_last_name)

In [38]:
session.execute("DROP TABLE IF EXISTS song_details")
session.execute("DROP TABLE IF EXISTS artist_details")
session.execute("DROP TABLE IF EXISTS user_details")

In [ ]:
session.shutdown()
cluster.shutdown()